The alignment from phonetisaurus is quite good, but there are some things that don't quite fit with an English speaker's intuition, so we repair some of them.

In [ ]:
_ngram_replacements = """
e}_ l}el	e|l}el
e}_ d}ed	e|d}ed
e}iy1 e}_	e|e}iy1
i}ix o|n}n	i|o}ix n}n
r}_ t|-}r t}t	r}r t|-|t}t
"""
ngram_replacements = {a.split('\t')[0]: a.split('\t')[1] for a in _ngram_replacements.split('\n') if '\t' in a}

In [ ]:
ngram_replacements

{'e}_ d}ed': 'e|d}ed',
 'e}_ l}el': 'e|l}el',
 'e}iy1 e}_': 'e|e}iy1',
 'i}ix o|n}n': 'i|o}ix n}n',
 'r}_ t|-}r t}t': 'r}r t|-|t}t'}

In [ ]:
def make_replacements(fixables):
  out = {}
  for line in fixables.split('\n'):
    if line == '':
      continue
    tmp = line.split('\t')
#    out[tmp[0]] = (tmp[1], tmp[2])
  return out

In [ ]:
bg_to_ug = make_pair_replacements(bigrams_to_unigrams)

In [ ]:
bg_to_ug

{'e}_': ('d}ed', 'e|d}ed'),
 'e}iy1': ('e}_', 'e|e}iy1'),
 'i}ix': ('o|n}n', 'i|o}ix')}

In [ ]:
def fix_graphones(graphones):
  out = []
  skip = False
  for i in range(len(graphones)):
    if skip:
      skip = False
      continue
    if graphones[i] in bg_to_ug:
      possible_replacement = bg_to_ug[graphones[i]]
      if i < len(graphones) and graphones[i+1] == possible_replacement[0]:
        out.append(possible_replacement[1])
        skip = True
    elif graphones[i] == '-|c}ao':
      out.append('-}_')
      out.append('c}k')
    elif graphones[i] == 'c}ao':
      if graphones[0] == 'n}n':
        out.append('c}s')
      else:
        out.append('c}k')
    elif graphones[i].startswith('-|'):
      out.append('-}_')
      out.append(graphones[i][2:])
    else:
      out.append(graphones[i])
  return out

In [ ]:
#fix_graphones("w}w e}iy1 e}_ w}w".split(' '))
fix_graphones("-|u}ah1 p}p".split(' '))


['-}_', 'u}ah1', 'p}p']

In [ ]:
with open('TIMIT.clean.corpus', 'r') as f:
  for line in f.readlines():
     graphones = line.split(' ')
     for i in range(0, len(graphones)):
       print (f'{graphones}: {graphones[i]}')